# Paperspace Gradient: TensorFlow 2 Quick Start for Beginners
Last modified: Oct 28th 2021

This demonstrates TensorFlow 2 usage in a Gradient Notebook. The material is based on the original TensorFlow 2 Beginner Quick Start at https://www.tensorflow.org/tutorials/quickstart/beginner .

See the end of the notebook for the original copyright notice and license.

This short introduction uses [Keras](https://www.tensorflow.org/guide/keras/overview) to:

1. Build a neural network that classifies images
2. Train this neural network
3. And, finally, evaluate the accuracy of the model

## Setup and GPUs

TensorFlow 2 is already installed in our container, so we can import it directly.

In [16]:
import tensorflow as tf

Provided that you have started this Notebook on a [Gradient GPU instance](https://docs.paperspace.com/gradient/more/instance-types), Gradient will see the GPU and TensorFlow will utilize it. Using a GPU provides large speedups for many machine learning models.

We can list the available GPUs.

In [17]:
gpus = tf.config.list_physical_devices('GPU')
gpus

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

By default, TensorFlow allocates all the memory on a GPU to the model being run. This is fine, until, for example, you run a container like this one that has more than one `.ipynb` notebook on it. Then the second notebook's model may fail due to the GPU being out of memory.

We can help with this by setting the memory used to [grow as needed](https://www.tensorflow.org/guide/gpu#limiting_gpu_memory_growth) rather than being allocated all at once at the start.

In [18]:
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

1 Physical GPUs, 1 Logical GPUs


## Load and prepare the data

The model shows classification of the handwritten digits 0-9 in the [MNIST dataset](http://yann.lecun.com/exdb/mnist/).

The data are loaded, converted from integers to floating-point numbers, and normalized.

In [19]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

## Build and train the model

The model is a simple single dense layer neural network with 10 output classes. We build it using `tf.keras.Sequential` by stacking layers.

Here we separate the model from the step to convert its outputs into probabilities.

In [20]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10)
])

For each example the model returns a vector of "[logits](https://developers.google.com/machine-learning/glossary#logits)" or "[log-odds](https://developers.google.com/machine-learning/glossary#log-odds)" scores, one for each class.

In [21]:
predictions = model(x_train[:1]).numpy()
predictions

array([[-0.3020117 ,  0.5922008 , -0.41980827, -0.24884853, -0.8993958 ,
         0.25624874,  0.20577453,  0.27147472, -0.08960623,  0.08015168]],
      dtype=float32)

The `tf.nn.softmax` function converts these logits to "probabilities" for each class.

In [22]:
tf.nn.softmax(predictions).numpy()

array([[0.07233316, 0.17688417, 0.06429527, 0.07628267, 0.03980126,
        0.12641159, 0.1201894 , 0.12835103, 0.08945073, 0.10600073]],
      dtype=float32)

(Note: It is possible to bake this `tf.nn.softmax` in as the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to
provide an exact and numerically stable loss calculation for all models when using a softmax output.)

To train the network, we choose an optimizer and loss function for training.

The `losses.SparseCategoricalCrossentropy` loss takes a vector of logits and a `True` index and returns a scalar loss for each example.

In [23]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the true class:
it is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.math.log(1/10) ~= 2.3`.

In [24]:
loss_fn(y_train[:1], predictions).numpy()

2.0682123

The optimized is the commonly used Adam algorithm.

In [25]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

The `Model.fit` method trains the model, adjusting the model parameters to minimize the loss.

In [26]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2946 - accuracy: 0.9143
Epoch 2/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.1420 - accuracy: 0.9576
Epoch 3/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.1077 - accuracy: 0.9669
Epoch 4/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0868 - accuracy: 0.9730
Epoch 5/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0751 - accuracy: 0.9766


## Evaluate the model

The `Model.evaluate` method checks the model's performance, usually on a "[Validation-set](https://developers.google.com/machine-learning/glossary#validation-set)" or "[Test-set](https://developers.google.com/machine-learning/glossary#test-set)".

In [27]:
model.evaluate(x_test, y_test, verbose=2)

313/313 - 0s - loss: 0.0778 - accuracy: 0.9759


[0.07781801372766495, 0.9758999943733215]

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the [TensorFlow tutorials](https://www.tensorflow.org/tutorials/).

## Output predictions

If you want your model to return a probability, you can wrap the trained model, and attach the softmax layer to it.

In [28]:
probability_model = tf.keras.Sequential([
    model,
    tf.keras.layers.Softmax()
])

In [29]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[9.05479638e-08, 2.39971225e-08, 7.25329392e-06, 5.21020374e-05,
        9.41129011e-13, 1.35973721e-07, 2.88683779e-15, 9.99939084e-01,
        1.32286630e-07, 1.18476009e-06],
       [6.62377460e-08, 8.56510131e-04, 9.98965383e-01, 6.89985391e-05,
        1.59924808e-13, 1.07135726e-04, 8.64839507e-08, 1.38364136e-12,
        1.86673992e-06, 1.81718200e-11],
       [5.31409682e-07, 9.98629928e-01, 9.91037086e-05, 8.31386660e-06,
        1.38301475e-04, 8.96011079e-06, 7.86305372e-06, 1.01774547e-03,
        8.63065870e-05, 3.13829537e-06],
       [9.99995470e-01, 5.50283014e-11, 2.01559897e-06, 1.39250972e-07,
        7.29438066e-10, 3.80786986e-07, 4.09131104e-07, 9.36438084e-07,
        5.92850269e-10, 7.34547086e-07],
       [2.30139221e-06, 3.46859674e-09, 5.36648940e-06, 1.20531212e-07,
        9.94976640e-01, 8.31843323e-08, 2.32134653e-07, 1.28873784e-04,
        1.43077855e-07, 4.88630775e-03]], dtype=float32)>

## Next steps

To proceed with TensorFlow 2 in Gradient, you can:
    
 - Try out the quick_start_advanced.ipynb notebook in this same container if you want to see some more advanced usage
 - Look at other Gradient material, such as the [tutorials](https://docs.paperspace.com/gradient/get-started/tutorials-list), [ML Showcase](https://ml-showcase.paperspace.com), [blog](https://blog.paperspace.com), or [community](https://community.paperspace.com)
 - Start writing your own projects, using our [documentation](https://docs.paperspace.com/gradient) when needed
 
If you get stuck or need help, [contact support](https://support.paperspace.com), and we will be happy to assist.

Good luck!

## Original TensorFlow copyright notice and license
##### Copyright 2019 The TensorFlow Authors.

In [30]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.